## Heart Disease Prediction (Binary Classification)

An attempt to build a machine learning model capable of predicting (based on their medical attributes) whether someone has heart disease or not.

**The approach:**
1. Problem Definition
2. Data Analysis:
  - Data
  - Evaluation
  - Features
3. Modeling
4. Experiment

---

## 1. Problem Definition

In a statement,
> Given medical attributes (clinical parameters) of a patient, can we predict whether that patient has heart disease or not?

---

## 2. Data Analysis

#### 2.1 Data

**Original Data is from:** https://archive.ics.uci.edu/ml/datasets/heart+disease

**We used the dataset of:** https://www.kaggle.com/ronitf/heart-disease-uci

**Attribute Information:**

1. age
2. sex
3. chest pain type (4 values)
4. resting blood pressure
5. serum cholestoral in mg/dl
6. fasting blood sugar > 120 mg/dl
7. resting electrocardiographic results (values 0,1,2)
8. maximum heart rate achieved
9. exercise induced angina
10. oldpeak = ST depression induced by exercise relative to rest
11. the slope of the peak exercise ST segment
12. number of major vessels (0-3) colored by flourosopy
13. thal: 3 = normal; 6 = fixed defect; 7 = reversable defect

#### 2.3 Features

1. **age** - age in years
2. **sex** - (1 = male; 0 = female)
3. **cp** - chest pain type
  - 0: Typical angina: chest pain related decrease blood supply to the heart
  - 1: Atypical angina: chest pain not related to heart
  - 2: Non-anginal pain: typically esophageal spasms (non heart related)
  - 3: Asymptomatic: chest pain not showing signs of disease
4. **trestbps** - resting blood pressure (in mm Hg on admission to the hospital) anything above 130-140 is typically cause for concern
5. **chol** - serum cholestoral in mg/dl
  - serum = LDL + HDL + .2 * triglycerides
  - above 200 is cause for concern
6. **fbs** - (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
  - '>126' mg/dL signals diabetes
7. **restecg** - resting electrocardiographic results
  - 0: Nothing to note
  - 1: ST-T Wave abnormality
    * can range from mild symptoms to severe problems
    * signals non-normal heart beat
  - 2: Possible or definite left ventricular hypertrophy
    * Enlarged heart's main pumping chamber
8. **thalach** - maximum heart rate achieved
9. **exang** - exercise induced angina (1 = yes; 0 = no)
10. **oldpeak** - ST depression induced by exercise relative to rest looks at stress of heart during excerci
11. **slope** - the slope of the peak exercise ST segment
  - 0: Upsloping: better heart rate with excercise (uncommon)
  - 1: Flatsloping: minimal change (typical healthy heart)
  - 2: Downslopins: signs of unhealthy heart
12. **ca** - number of major vessels (0-3) colored by flourosopy
  - colored vessel means the doctor can see the blood passing through
  - the more blood movement the better (no clots)
13. **thal** - thalium stress result
  - 1,3: normal
  - 6: fixed defect: used to be defect but ok now
  - 7: reversable defect: no proper blood movement when excercising
14. **target** - have disease or not (1=yes, 0=no) (= the predicted attribute)